#### LangChain Essentials Course

# Getting Started with LangChain

LangChain is one of the most popular open source libraries for AI Engineers. It's goal is to abstract away the complexity in building AI software, provide easy-to-use building blocks, and make it easier when switching between AI service providers.

In this example, we will introduce LangChain, building a simple LLM-powered assistant. We'll provide examples for both OpenAI's `gpt-4o-mini` *and* Meta's `llama3.2` via Ollama!

## Choosing your Model

This example is split into two versions, this version uses OpenAI - providing **S**tate **o**f **t** **A**rt performance via their API. If you'd prefer to run everything locally you can try the [Ollama version](https://github.com/aurelio-labs/agents-course/blob/main/04-langchain-ecosystem/01-langchain-essentials/01-langchain-intro-ollama.ipynb).

First we need to settup our imports to make sure we have both models we want to test.

In [835]:
# need to add llama here too.
import openai

# OpenAI key to use the model
OPENAI_API_KEY = ""
openai.api_key = OPENAI_API_KEY

*ollama pull llama3.2:1b-instruct-fp16* - Use in terminal to download correct model we are using.

In [836]:
from langchain_ollama.chat_models import ChatOllama

# Here we can list a couple of models to choose from.
openai_model = "gpt-4o-mini"
llama_model = "llama3.2:1b-instruct-fp16"

# Choose between which model you would like to use
active_model = openai_model

So the task at hand is to add any lyrics into the 'song' variable, then we want to create three core pieces from this song:

1. Song title
2. Song description
3. One additional verse in the song.

We will make all of this using a sequential chain, and then we will display it as:

> Original song:

> New song name:

> New song description:

> New verse:

> Where the new verse should be played in the song:

Here we can input our song to start us of with, currently this is using Viva La Vida by Coldplay

In [837]:
song = """
\
I used to rule the world \
Seas would rise when I gave the word \
Now in the morning, I sleep alone \
Sweep the streets I used to own \
I used to roll the dice \
Feel the fear in my enemy's eyes \
Listen as the crowd would sing \
Now the old king is dead, long live the king \
One minute, I held the key \
Next the walls were closed on me \
And I discovered that my castles stand \
Upon pillars of salt and pillars of sand \
\
I hear Jerusalem bells a-ringin' \
Roman Cavalry choirs are singin' \
Be my mirror, my sword and shield \
My missionaries in a foreign field \
For some reason, I can't explain \
Once you'd gone, there was never, never an honest word \
And that was when I ruled the world \
\
It was a wicked and wild wind \
Blew down the doors to let me in \
Shattered windows and the sound of drums \
People couldn't believe what I'd become \
Revolutionaries wait \
For my head on a silver plate \
Just a puppet on a lonely string \
Aw, who would ever wanna be king? \
\
I hear Jerusalem bells a-ringin' \
Roman Cavalry choirs are singing \
Be my mirror, my sword and shield \
My missionaries in a foreign field \
For some reason, I can't explain \
I know Saint Peter won't call my name \
Never an honest word \
But that was when I ruled the world \
\
Oh-oh-oh, oh-oh, oh \
Oh-oh-oh, oh-oh, oh \
Oh-oh-oh, oh-oh, oh \
Oh-oh-oh, oh-oh, oh \
Oh-oh-oh, oh-oh, oh \
\
I hear Jerusalem bells a-ringin' \
Roman Cavalry choirs are singin' \
Be my mirror, my sword and shield \
My missionaries in a foreign field \
For some reason I can't explain \
I know Saint Peter won't call my name \
Never an honest word \
But that was when I ruled the world \
"""

Now we want to import all our libaries needed for using LangChain's sequential chain.

In [838]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

Here we will make a LLM able to connect to our models.

In [839]:
if active_model == llama_model:
    llm = ChatOllama(temperature=0.0, model=llama_model)
    creative_llm = ChatOllama(temperature=0.9, model=llama_model)
elif active_model == openai_model:
    llm = ChatOpenAI(temperature=0.0, model=openai_model, openai_api_key = OPENAI_API_KEY)
    creative_llm = ChatOpenAI(temperature=0.9, model=openai_model, openai_api_key = OPENAI_API_KEY)
else:
    raise ValueError("You need to apply the correct name to the model you are referring to")

ValidationError: 1 validation error for ChatOpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'temperature': 0.0, 'ope...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

Now we will get into our chain, for the sequential chains it is important to note everything is done within a string, and not an f string, and keep an eye on how variables are passed in and out, as if these get messed up it won't create an error here and instead create an error at the overall chain...

In [827]:
# Defining the system prompt (how the AI should act)
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant that helps generate song titles."
)

# Defining the user prompt (what we want the AI to do)
user_prompt = HumanMessagePromptTemplate.from_template(
    "Can you make a title for this song:"
    "\n\n{song}"
)

# prompt template 1: making a song title
first_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

# chain 1: inputs: song / outputs: song_title
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="song_title"
                    )

In [828]:
print(first_prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an AI assistant that helps generate song titles.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['song'], input_types={}, partial_variables={}, template='Can you make a title for this song:\n\n{song}'), additional_kwargs={})]


In [829]:
# prompt template 2: making a song description
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following song in 2 sentences (max 40 words) using the song title in the summary:"
    "\n\n{song}\n\n{song_title}"
)
# chain 2: inputs: song, song_title / outputs: summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )

In [830]:
print(second_prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['song', 'song_title'], input_types={}, partial_variables={}, template='Can you summarize the following song in 2 sentences (max 40 words) using the song title in the summary:\n\n{song}\n\n{song_title}'), additional_kwargs={})]


In [831]:
# prompt template 3: creating a new verse
third_prompt = ChatPromptTemplate.from_template(
    "Can make one short verse for this song:\n\n{song}"
)
# chain 3: inputs: song / output: new_verse
chain_three = LLMChain(llm=creative_llm, prompt=third_prompt,
                       output_key="new_verse"
                      )

In [832]:
# prompt template 4: adding the new verse to the original music
fourth_prompt = ChatPromptTemplate.from_template(
    "where should the {new_verse} be played in the song {song}, \
        explicitly say which line of the original song this new verse should come afterwards"
)
# chain 3: inputs: song, new_verse / outputs: new_song
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                       output_key="new_song"
                      )

Now we have all our chains ready we can make a 'overall' chain, this will bind all our chains together and is critical for any of this to work.

In [833]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],                 #   all the previous chains we just made
    input_variables=["song"],                                               #   where our song will be inputted
    output_variables=["song_title", "summary","new_verse","new_song"],      #   all the outputs we create
    verbose=True                                                            #   to show AI trail of thought
)

Now we are all settup we can call our overall_chain like a function, passing in only one variable song!

In [834]:
overall_chain(song)



> Entering new SequentialChain chain...



> Finished chain.


{'song': "\nI used to rule the world Seas would rise when I gave the word Now in the morning, I sleep alone Sweep the streets I used to own I used to roll the dice Feel the fear in my enemy's eyes Listen as the crowd would sing Now the old king is dead, long live the king One minute, I held the key Next the walls were closed on me And I discovered that my castles stand Upon pillars of salt and pillars of sand I hear Jerusalem bells a-ringin' Roman Cavalry choirs are singin' Be my mirror, my sword and shield My missionaries in a foreign field For some reason, I can't explain Once you'd gone, there was never, never an honest word And that was when I ruled the world It was a wicked and wild wind Blew down the doors to let me in Shattered windows and the sound of drums People couldn't believe what I'd become Revolutionaries wait For my head on a silver plate Just a puppet on a lonely string Aw, who would ever wanna be king? I hear Jerusalem bells a-ringin' Roman Cavalry choirs are singing 